## weight-tuning_EB_LG
*   Epoch Bound : 在train 4000中
*   Learning Goal : loss要小於0.25

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the Boston Housing dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/新型學習演算法/diabetes.csv')

In [ ]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
print("dimension of data: {}".format(data.shape))

dimension of data: (768, 9)


In [ ]:
# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(data.drop('Outcome',axis = 1))
y = data['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# 將數據轉換為 PyTorch 張量
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [ ]:
# 設定參數
D_in = X.shape[1]
H = 11
D_out = 1
print(f'Input layer: {D_in}')

# learning rate
learning_rate = 0.001

# learning goal
lg = 0.32

epoch_bound = 5000

Input layer: 8


In [ ]:
from collections import OrderedDict
# Define 2-layer neural network
model_EB_LG = torch.nn.Sequential(OrderedDict([
    ('fc1', torch.nn.Linear(D_in, H)),
    ('relu', torch.nn.ReLU()),
    ('output', torch.nn.Linear(H, D_out))
])
)

In [ ]:
import time
# 損失函數和learning rate
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for binary classification without sigmoid
optimizer = optim.Adam(model_EB_LG.parameters(), lr=learning_rate)

train_losses = []

# Train the network
loss = float('Inf')
start_time = time.time()
for epoch in range(epoch_bound):
  if loss > lg:

    output = model_EB_LG(X_train)
    loss = criterion(output.view(-1), y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Collect the training loss values in list
    train_losses.append(loss.item())
  else:
    break


end_time = time.time()
duration = end_time - start_time
print(f"Training took {duration} seconds.")
print()
if epoch+1 < epoch_bound:
  print('Acceptable, Epoch {}, Loss: {}'.format(epoch+1, loss.item()))
else:
  print('Unacceptable, reach epochs {} but loss > {}.\nEpoch {}, Loss: {}'.format(epoch_bound, lg, epoch+1, loss.item()))



Training took 3.0064640045166016 seconds.

Unacceptable, reach epochs 5000 but loss > 0.32.
Epoch 5000, Loss: 0.3387976884841919


In [ ]:
y_pred = model_EB_LG(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = torch.round(torch.sigmoid(y_pred))

# Calculate accuracy
total = len(y_test)
correct = 0
for i in range(len(y_pred_labels)):
  if y_pred_labels[i] == y_test[i]:
    correct += 1
accuracy = (correct / total) * 100

print(f"Accuracy: {accuracy}")

Accuracy: 72.07792207792207


## Clean the data and retrain it

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
def count_zero_num(feature):
  return len(data[data[feature]==0])
print('zero-values count')
for i in data.columns:
  temp = count_zero_num(i)
  print(f'{i}:\n{temp}\n')

zero-values count
Pregnancies:
111

Glucose:
5

BloodPressure:
35

SkinThickness:
227

Insulin:
374

BMI:
11

DiabetesPedigreeFunction:
0

Age:
0

Outcome:
500



### 把Pregnancies, SkinThickness, Insulin 0 的資料用mean取代

In [ ]:
data[['Pregnancies', 'SkinThickness', 'Insulin']] = data[['Pregnancies', 'SkinThickness', 'Insulin']].replace(0, np.nan)

In [ ]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148,72,35.0,NaN,33.6,0.627,50,1
1,1.0,85,66,29.0,NaN,26.6,0.351,31,0
2,8.0,183,64,NaN,NaN,23.3,0.672,32,1
3,1.0,89,66,23.0,94.0,28.1,0.167,21,0
4,NaN,137,40,35.0,168.0,43.1,2.288,33,1


In [ ]:
data[['Pregnancies', 'SkinThickness', 'Insulin']] = data[['Pregnancies', 'SkinThickness', 'Insulin']].fillna(data[['Pregnancies', 'SkinThickness', 'Insulin']].mean())
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.000000,148,72,35.00000,155.548223,33.6,0.627,50,1
1,1.000000,85,66,29.00000,155.548223,26.6,0.351,31,0
2,8.000000,183,64,29.15342,155.548223,23.3,0.672,32,1
3,1.000000,89,66,23.00000,94.000000,28.1,0.167,21,0
4,4.494673,137,40,35.00000,168.000000,43.1,2.288,33,1


In [ ]:
for i in data.columns:
  temp = count_zero_num(i)
  print(f'{i}:\n{temp}\n')

Pregnancies:
0

Glucose:
5

BloodPressure:
35

SkinThickness:
0

Insulin:
0

BMI:
11

DiabetesPedigreeFunction:
0

Age:
0

Outcome:
500



In [ ]:
# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(data.drop('Outcome',axis = 1))
y = data['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# 將數據轉換為 PyTorch 張量
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [ ]:
from collections import OrderedDict
# Define 2-layer neural network
model_EB_LG = torch.nn.Sequential(OrderedDict([
    ('fc1', torch.nn.Linear(D_in, H)),
    ('relu', torch.nn.ReLU()),
    ('output', torch.nn.Linear(H, D_out))
])
)

# 損失函數和learning rate
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for binary classification without sigmoid
optimizer = optim.Adam(model_EB_LG.parameters(), lr=learning_rate)

train_losses = []

# Train the network
loss = float('Inf')
start_time = time.time()
for epoch in range(epoch_bound):
  if loss > lg:

    output = model_EB_LG(X_train)
    loss = criterion(output.view(-1), y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Collect the training loss values in list
    train_losses.append(loss.item())
  else:
    break


end_time = time.time()
duration = end_time - start_time
print(f"Training took {duration} seconds.")
print()
if epoch+1 < epoch_bound:
  print('Acceptable, Epoch {}, Loss: {}'.format(epoch+1, loss.item()))
else:
  print('Unacceptable, reach epochs {} but loss > {}.\nEpoch {}, Loss: {}'.format(epoch_bound, lg, epoch+1, loss.item()))



Training took 3.5471599102020264 seconds.

Unacceptable, reach epochs 5000 but loss > 0.32.
Epoch 5000, Loss: 0.35440340638160706


In [ ]:
y_pred = model_EB_LG(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = torch.round(torch.sigmoid(y_pred))

# Calculate accuracy
total = len(y_test)
correct = 0
for i in range(len(y_pred_labels)):
  if y_pred_labels[i] == y_test[i]:
    correct += 1
accuracy = (correct / total) * 100

print(f"Accuracy: {accuracy}")

Accuracy: 72.07792207792207


## Find unacceptable cases
Two-class classification application
Use learing_goal_2 :


1.   If f(x,w) ≥ 𝛆, label=1
2.   If f(x,w) ≤ -𝛆, label=0
3.   If 𝛆 < f(x,w) < 1-𝛆, then x unknown



In [ ]:
𝛆 = 2.5
acceptable = []
unacceptable = []
acceptable_idx = []
unacceptable_idx = []
unknown = []
unknown_idx = []
for i in range(len(X_train)):
  output = model_EB_LG(X_train[i])
  if output >= 𝛆:
    if torch.round(torch.sigmoid(output)) == y_train[i]:
      acceptable.append(X_train[i])
      acceptable_idx.append(i)
    else:
      unacceptable.append(X_train[i])
      unacceptable_idx.append(i)
  elif output <= -𝛆:
    if torch.round(torch.sigmoid(output)) == y_train[i]:
      acceptable.append(X_train[i])
      acceptable_idx.append(i)
    else:
      unacceptable.append(X_train[i])
      unacceptable_idx.append(i)
  else:
    unknown.append(X_train[i])
    unknown_idx.append(i)
print(len(unacceptable))

6


In [ ]:
unacceptable_idx

[46, 200, 445, 528, 542, 563]

In [ ]:
len(X_train) == len(unknown)+len(acceptable)+len(unacceptable)

True

In [ ]:
len(unknown)

366

## Isolating_Ri_ZG

In [ ]:
# random generate gama (8-dimensional tensor)
𝛾 = np.random.rand(8)
k = unacceptable[0]
for i in acceptable:
  if np.dot(𝛾, (i-k)) == 0:
    print('unacceptable 𝛾')
    break
print(f'This 𝛾={𝛾} is acceptable')

This 𝛾=[0.61858168 0.31749122 0.58855339 0.28924314 0.79256803 0.10949156
 0.59944026 0.35880042] is acceptable


In [ ]:
type(𝛾)

numpy.ndarray

In [ ]:
import random
random_float = random.uniform(0, 1)
print(random_float)

0.568692343624573


In [ ]:
import random
flag = 1
while flag == 1:
  𝜁 = random.uniform(0, 1)
  for i in acceptable:
    if (𝜁 + np.dot(𝛾, (i-k)))*(𝜁 - np.dot(𝛾, (i-k))) >= 0:
      '''print('unacceptable 𝜁')'''
      break
    if i.tolist() == acceptable[-1].tolist():
      print(f'This 𝜁={𝜁} is acceptable')
      flag = 0

This 𝜁=2.0414911916954992e-05 is acceptable


## 看要改幾個unacceptable case才可以達到learning goal

In [ ]:
while len(unacceptable_idx) > 0:
  for i in range(0,1):
    # 從第一個unacceptable case開始修正
    new_node_weights = 𝛾 # 初始化input to hidden nodes的weights
    new_node_bias_1 = 𝜁 - np.dot(𝛾, unacceptable[i])
    new_node_bias_2 = -np.dot(𝛾, unacceptable[i])
    new_node_bias_3 = -𝜁 - np.dot(𝛾, unacceptable[i])
    # activation of i hidden node corresponding to x_k
    hidden_layer_value = torch.matmul(unacceptable[0], model_EB_LG.fc1.weight.T)
    hidden_layer_Relu = torch.where(hidden_layer_value >= 0, hidden_layer_value, torch.zeros_like(hidden_layer_value)) # ReLU
    new_node_output_weights_1 = (y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T))/𝜁
    new_node_output_weights_2 = (-2*(y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T)))/𝜁
    new_node_output_weights_3 = (y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T))/𝜁
    '''print(new_node_weights)
    print(new_node_bias)
    print(new_node_output_weights)'''

    # 手動新增hidden node按照數學公式
    hidden_layer_params = model_EB_LG.fc1.weight.detach().numpy()
    hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
    hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
    hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
    '''print(hidden_layer_params)'''
    bias_matrix = model_EB_LG.fc1.bias.detach().numpy()
    new_bias = np.append(bias_matrix, new_node_bias_1)
    new_bias = np.append(new_bias, new_node_bias_2)
    new_bias = np.append(new_bias, new_node_bias_3)
    '''print(new_bias)'''
    weight_matrix_2 = model_EB_LG.output.weight.detach().numpy()
    modified_weight_matrix_2 = np.append(weight_matrix_2[0], new_node_output_weights_1)
    modified_weight_matrix_2 = np.append(modified_weight_matrix_2, new_node_output_weights_2)
    modified_weight_matrix_2 = np.append(modified_weight_matrix_2, new_node_output_weights_3)
    # Get the number of hidden nodes in the second layer
    h = model_EB_LG.fc1.bias.shape
    updated_weight_matrix_2 = np.empty((1,h[0]+(i+1)*3))
    updated_weight_matrix_2[0] = modified_weight_matrix_2
    '''print(updated_weight_matrix_2)'''
    # 將自訂的權重和偏差儲存在模型中
    model_EB_LG.fc1.weight = nn.Parameter(torch.from_numpy(hidden_layer_params).type(torch.Tensor))
    model_EB_LG.fc1.bias = nn.Parameter(torch.from_numpy(new_bias).type(torch.Tensor))
    model_EB_LG.output.weight = nn.Parameter(torch.from_numpy(updated_weight_matrix_2).type(torch.Tensor))

    print(f'調整第{i+1}個unacceptable case後的parameters setting')
    for name, param in model_EB_LG.named_parameters():
      print(name, param)
    output = model_EB_LG(X_train)
    loss = criterion(output.view(-1), y_train)
    print()
    print('loss = {}'.format(loss))
    print('==========================================================================')

    if loss <= lg:
      print('Acceptable model')
      break
    else:
      print('Fail')

    acceptable = []
    unacceptable = []
    acceptable_idx = []
    unacceptable_idx = []
    unknown = []
    unknown_idx = []
    for i in range(len(X_train)):
      output = model_EB_LG(X_train[i])
      if output >= 𝛆:
        if torch.round(torch.sigmoid(output)) == y_train[i]:
          acceptable.append(X_train[i])
          acceptable_idx.append(i)
        else:
          unacceptable.append(X_train[i])
          unacceptable_idx.append(i)
      elif output <= -𝛆:
        if torch.round(torch.sigmoid(output)) == y_train[i]:
          acceptable.append(X_train[i])
          acceptable_idx.append(i)
        else:
          unacceptable.append(X_train[i])
          unacceptable_idx.append(i)
      else:
        unknown.append(X_train[i])
        unknown_idx.append(i)
    print('unacceptable cases count:',len(unacceptable))
    print(unacceptable_idx)
    print()

調整第1個unacceptable case後的parameters setting
fc1.weight Parameter containing:
tensor([[ 1.6545,  0.2266, -0.2436,  1.1326,  1.8542, -0.9023,  0.3489, -1.3881],
        [-0.2383, -0.2281, -0.3418, -0.2723,  0.5003,  0.2720, -0.2217, -1.1496],
        [ 0.5796,  0.3203, -0.8478,  1.3130, -2.1655,  0.1488, -0.5279,  0.7556],
        [ 0.0089, -0.5108, -0.4134,  0.8098, -0.9616, -0.4552,  0.0476, -0.4634],
        [ 0.6121,  0.2196,  0.5672,  1.0401,  1.8232, -0.5801, -0.2609, -2.1568],
        [-1.0099,  0.2423,  0.4430, -0.3922, -0.2467,  0.2342, -0.5985, -0.4662],
        [-1.0488,  0.9809,  0.2157,  0.1133,  0.0317,  0.3678, -0.8338, -0.7840],
        [ 0.4011,  0.1203, -0.8790,  1.5036, -0.9759, -0.1380,  0.3679, -0.3186],
        [ 0.1670, -0.5466,  0.8542, -0.5203, -0.0713, -0.7620,  0.0515,  0.0359],
        [-0.2758, -0.3958, -0.8232, -0.7038,  0.0147, -0.4219,  0.1434,  0.0926],
        [-1.1390,  0.4951,  0.0478,  0.1154, -0.9939, -0.5608, -0.2536, -0.4804],
        [ 0.6186,  0.3

## X_text accuracy

In [ ]:
y_pred = model_EB_LG(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = torch.round(torch.sigmoid(y_pred))

# Calculate accuracy
total = len(y_test)
correct = 0
for i in range(len(y_pred_labels)):
  if y_pred_labels[i] == y_test[i]:
    correct += 1
accuracy = (correct / total) * 100

print(f"Accuracy: {accuracy}")


Accuracy: 72.07792207792207


## Analogizing_Ri_LG2_SO_MU
### 從第一個unacceptable case開始修正

In [ ]:
for i in range(0,1):
  # 從第一個unacceptable case開始修正
  new_node_weights = 𝛾 # 初始化input to hidden nodes的weights
  new_node_bias_1 = 𝜁 - np.dot(𝛾, unacceptable[i])
  new_node_bias_2 = -np.dot(𝛾, unacceptable[i])
  new_node_bias_3 = -𝜁 - np.dot(𝛾, unacceptable[i])
  # activation of i hidden node corresponding to x_k
  hidden_layer_value = torch.matmul(unacceptable[0], model_EB_LG.fc1.weight.T)
  hidden_layer_Relu = torch.where(hidden_layer_value >= 0, hidden_layer_value, torch.zeros_like(hidden_layer_value)) # ReLU
  new_node_output_weights_1 = (y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T))/𝜁
  new_node_output_weights_2 = (-2*(y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T)))/𝜁
  new_node_output_weights_3 = (y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T))/𝜁
  '''print(new_node_weights)
  print(new_node_bias)
  print(new_node_output_weights)'''

  # 手動新增hidden node按照數學公式
  hidden_layer_params = model_EB_LG.fc1.weight.detach().numpy()
  hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
  hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
  hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
  '''print(hidden_layer_params)'''
  bias_matrix = model_EB_LG.fc1.bias.detach().numpy()
  new_bias = np.append(bias_matrix, new_node_bias_1)
  new_bias = np.append(new_bias, new_node_bias_2)
  new_bias = np.append(new_bias, new_node_bias_3)
  '''print(new_bias)'''
  weight_matrix_2 = model_EB_LG.output.weight.detach().numpy()
  modified_weight_matrix_2 = np.append(weight_matrix_2[0], new_node_output_weights_1)
  modified_weight_matrix_2 = np.append(modified_weight_matrix_2, new_node_output_weights_2)
  modified_weight_matrix_2 = np.append(modified_weight_matrix_2, new_node_output_weights_3)
  updated_weight_matrix_2 = np.empty((1,H+(i+1)*3))
  updated_weight_matrix_2[0] = modified_weight_matrix_2
  '''print(updated_weight_matrix_2)'''
  # 將自訂的權重和偏差儲存在模型中
  model_EB_LG.fc1.weight = nn.Parameter(torch.from_numpy(hidden_layer_params).type(torch.Tensor))
  model_EB_LG.fc1.bias = nn.Parameter(torch.from_numpy(new_bias).type(torch.Tensor))
  model_EB_LG.output.weight = nn.Parameter(torch.from_numpy(updated_weight_matrix_2).type(torch.Tensor))

  print(f'調整第{i+1}個unacceptable case後的parameters setting')
  for name, param in model_EB_LG.named_parameters():
    print(name, param)
  output = model_EB_LG(X_train)
  loss = criterion(output.view(-1), y_train)
  print('\n')
  print('loss = {}'.format(loss))
  print('==========================================================================')

if loss <= lg:
  print('Acceptable model')
else:
  print('Fail')

In [ ]:
acceptable = []
unacceptable = []
acceptable_idx = []
unacceptable_idx = []
unknown = []
unknown_idx = []
for i in range(len(X_train)):
  output = model_EB_LG(X_train[i])
  if output >= 𝛆:
    if torch.round(torch.sigmoid(output)) == y_train[i]:
      acceptable.append(X_train[i])
      acceptable_idx.append(i)
    else:
      unacceptable.append(X_train[i])
      unacceptable_idx.append(i)
  elif output <= -𝛆:
    if torch.round(torch.sigmoid(output)) == y_train[i]:
      acceptable.append(X_train[i])
      acceptable_idx.append(i)
    else:
      unacceptable.append(X_train[i])
      unacceptable_idx.append(i)
  else:
    unknown.append(X_train[i])
    unknown_idx.append(i)
print(len(unacceptable))

In [ ]:
unacceptable_idx

In [ ]:
acceptable = []
unacceptable = []
acceptable_idx = []
unacceptable_idx = []
unknown = []
unknown_idx = []
for i in range(len(X_train)):
  output = model_EB_LG(X_train[i])
  if output >= 𝛆:
    if torch.round(torch.sigmoid(output)) == y_train[i]:
      acceptable.append(X_train[i])
      acceptable_idx.append(i)
    else:
      unacceptable.append(X_train[i])
      unacceptable_idx.append(i)
  elif output <= -𝛆:
    if torch.round(torch.sigmoid(output)) == y_train[i]:
      acceptable.append(X_train[i])
      acceptable_idx.append(i)
    else:
      unacceptable.append(X_train[i])
      unacceptable_idx.append(i)
  else:
    unknown.append(X_train[i])
    unknown_idx.append(i)
print(len(unacceptable))

10


In [ ]:
unacceptable_idx

[64, 115, 156, 316, 324, 335, 351, 441, 475, 491]

### 用第二個unacceptable case修正model

In [ ]:
for i in range(0,1):
  # 從第一個unacceptable case開始修正
  new_node_weights = 𝛾 # 初始化input to hidden nodes的weights
  new_node_bias_1 = 𝜁 - np.dot(𝛾, unacceptable[i])
  new_node_bias_2 = -np.dot(𝛾, unacceptable[i])
  new_node_bias_3 = -𝜁 - np.dot(𝛾, unacceptable[i])
  # activation of i hidden node corresponding to x_k
  hidden_layer_value = torch.matmul(unacceptable[0], model_EB_LG.fc1.weight.T)
  hidden_layer_Relu = torch.where(hidden_layer_value >= 0, hidden_layer_value, torch.zeros_like(hidden_layer_value)) # ReLU
  new_node_output_weights_1 = (y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T))/𝜁
  new_node_output_weights_2 = (-2*(y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T)))/𝜁
  new_node_output_weights_3 = (y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T))/𝜁
  '''print(new_node_weights)
  print(new_node_bias)
  print(new_node_output_weights)'''

  # 手動新增hidden node按照數學公式
  hidden_layer_params = model_EB_LG.fc1.weight.detach().numpy()
  hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
  hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
  hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
  '''print(hidden_layer_params)'''
  bias_matrix = model_EB_LG.fc1.bias.detach().numpy()
  new_bias = np.append(bias_matrix, new_node_bias_1)
  new_bias = np.append(new_bias, new_node_bias_2)
  new_bias = np.append(new_bias, new_node_bias_3)
  '''print(new_bias)'''
  weight_matrix_2 = model_EB_LG.output.weight.detach().numpy()
  modified_weight_matrix_2 = np.append(weight_matrix_2[0], new_node_output_weights_1)
  modified_weight_matrix_2 = np.append(modified_weight_matrix_2, new_node_output_weights_2)
  modified_weight_matrix_2 = np.append(modified_weight_matrix_2, new_node_output_weights_3)
  updated_weight_matrix_2 = np.empty((1,H+(i+1)*3))
  updated_weight_matrix_2[0] = modified_weight_matrix_2
  '''print(updated_weight_matrix_2)'''
  # 將自訂的權重和偏差儲存在模型中
  model_EB_LG.fc1.weight = nn.Parameter(torch.from_numpy(hidden_layer_params).type(torch.Tensor))
  model_EB_LG.fc1.bias = nn.Parameter(torch.from_numpy(new_bias).type(torch.Tensor))
  model_EB_LG.output.weight = nn.Parameter(torch.from_numpy(updated_weight_matrix_2).type(torch.Tensor))

  print(f'調整第{i+1}個unacceptable case後的parameters setting')
  for name, param in model_EB_LG.named_parameters():
    print(name, param)
  output = model_EB_LG(X_train)
  loss = criterion(output.view(-1), y_train)
  print('\n')
  print('loss = {}'.format(loss))
  print('==========================================================================')

if loss <= lg:
  print('Acceptable model')
else:
  print('Fail')

調整第2個unacceptable case後的parameters setting
fc1.weight Parameter containing:
tensor([[-0.4294, -0.5413, -0.5504,  0.3812, -1.4949, -0.1351, -0.8694,  1.2358],
        [-1.0213, -0.3627,  0.1970, -0.4086,  0.2433, -0.3123, -0.1819, -1.4422],
        [-0.4053,  0.8297, -0.4579, -0.1478, -0.2852,  0.4405, -0.0337,  0.3097],
        [ 0.9144,  0.7410,  0.1643,  0.4742,  0.4254,  0.6694,  0.0473,  0.7995],
        [ 0.9144,  0.7410,  0.1643,  0.4742,  0.4254,  0.6694,  0.0473,  0.7995],
        [ 0.9144,  0.7410,  0.1643,  0.4742,  0.4254,  0.6694,  0.0473,  0.7995],
        [ 0.9144,  0.7410,  0.1643,  0.4742,  0.4254,  0.6694,  0.0473,  0.7995],
        [ 0.9144,  0.7410,  0.1643,  0.4742,  0.4254,  0.6694,  0.0473,  0.7995],
        [ 0.9144,  0.7410,  0.1643,  0.4742,  0.4254,  0.6694,  0.0473,  0.7995]],
       requires_grad=True)
fc1.bias Parameter containing:
tensor([ 0.4643, -0.0637,  0.7133,  1.4395,  1.4389,  1.4382,  0.9929,  0.9923,
         0.9916], requires_grad=True)
output.we

## 用第三個unacceptable case

In [ ]:
for i in range(2,11):
  # 從第一個unacceptable case開始修正
  new_node_weights = 𝛾 # 初始化input to hidden nodes的weights
  new_node_bias_1 = 𝜁 - np.dot(𝛾, unacceptable[i])
  new_node_bias_2 = -np.dot(𝛾, unacceptable[i])
  new_node_bias_3 = -𝜁 - np.dot(𝛾, unacceptable[i])
  # activation of i hidden node corresponding to x_k
  hidden_layer_value = torch.matmul(unacceptable[0], model_EB_LG.fc1.weight.T)
  hidden_layer_Relu = torch.where(hidden_layer_value >= 0, hidden_layer_value, torch.zeros_like(hidden_layer_value)) # ReLU
  new_node_output_weights_1 = (y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T))/𝜁
  new_node_output_weights_2 = (-2*(y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T)))/𝜁
  new_node_output_weights_3 = (y_train[unacceptable_idx[i]] - model_EB_LG.output.bias.detach().numpy() - np.dot(model_EB_LG.output.weight.detach().numpy(), hidden_layer_Relu.detach().numpy().T))/𝜁
  '''print(new_node_weights)
  print(new_node_bias)
  print(new_node_output_weights)'''

  # 手動新增hidden node按照數學公式
  hidden_layer_params = model_EB_LG.fc1.weight.detach().numpy()
  hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
  hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
  hidden_layer_params = np.vstack((hidden_layer_params, new_node_weights))
  '''print(hidden_layer_params)'''
  bias_matrix = model_EB_LG.fc1.bias.detach().numpy()
  new_bias = np.append(bias_matrix, new_node_bias_1)
  new_bias = np.append(new_bias, new_node_bias_2)
  new_bias = np.append(new_bias, new_node_bias_3)
  '''print(new_bias)'''
  weight_matrix_2 = model_EB_LG.output.weight.detach().numpy()
  modified_weight_matrix_2 = np.append(weight_matrix_2[0], new_node_output_weights_1)
  modified_weight_matrix_2 = np.append(modified_weight_matrix_2, new_node_output_weights_2)
  modified_weight_matrix_2 = np.append(modified_weight_matrix_2, new_node_output_weights_3)
  updated_weight_matrix_2 = np.empty((1,H+(i+1)*3))
  updated_weight_matrix_2[0] = modified_weight_matrix_2
  '''print(updated_weight_matrix_2)'''
  # 將自訂的權重和偏差儲存在模型中
  model_EB_LG.fc1.weight = nn.Parameter(torch.from_numpy(hidden_layer_params).type(torch.Tensor))
  model_EB_LG.fc1.bias = nn.Parameter(torch.from_numpy(new_bias).type(torch.Tensor))
  model_EB_LG.output.weight = nn.Parameter(torch.from_numpy(updated_weight_matrix_2).type(torch.Tensor))

  print(f'調整第{i+1}個unacceptable case後的parameters setting')
  for name, param in model_EB_LG.named_parameters():
    print(name, param)
  output = model_EB_LG(X_train)
  loss = criterion(output.view(-1), y_train)
  print('\n')
  print('loss = {}'.format(loss))
  print('==========================================================================')

if loss <= lg:
  print('Acceptable model')
else:
  print('Fail')